In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import spherical_jn
from scipy.integrate import quad
from tqdm import tqdm

from interaction import potential, LocalSystem
from counterterm import local_LO_counterterm
from constants import MU, BETA4
from utility import log_mesh, ft_matrix_gen

plt.style.use('science')

In [2]:
rgf = np.loadtxt('rgflow_local_lo_a0_fix_hp_n_14_coord_ir_corrected.txt')

In [3]:
rgf.shape

(95, 2)

In [4]:
ii = -1
R, c0 = rgf[ii]
print(f'{BETA4/R}')

def v4(r):
    if r == 0:
        return 0
    return potential(r, R)


def pot(r):
    return v4(r) + c0*local_LO_counterterm(r, R)


def f(r, p, k):
    if p == 0 and k == 0:
        return 2/np.pi * r**2 * pot(r)
    if p == 0:
        return 2/np.pi * 1/k * r * pot(r) * np.sin(k*r)
    if k == 0:
        return 2/np.pi * 1/p * np.sin(p*r) * pot(r) * r
    return 2/np.pi * 1/(p*k) * np.sin(p*r) * pot(r) * np.sin(k*r)


def g(z, p, k):
    return -1/z * np.sin(-p*np.log(z)) * pot(-np.log(z)) * np.sin(-k*np.log(z))


def h(y, p, k):
    return 2/np.pi * 1/(q[0]*q[-1]) * (1 + np.tan(y)**2) * np.sin(p*np.tan(y)) * pot(np.tan(y)) * np.sin(k*np.tan(y))


nq = 200
q, wq = log_mesh(0, 20*2/R, nq)

v = np.zeros((nq, nq))

78.18181818181819


In [5]:
%%time
result = quad(pot, 0, np.infty, weight='cos', wvar=q[0]-q[-1], full_output=1)

CPU times: user 4.89 ms, sys: 318 µs, total: 5.21 ms
Wall time: 5.09 ms


In [6]:
result

(-2.999916934113314e-11,
 2.7874591943071358e-09,
 {'neval': 650,
  'lst': 6,
  'rslst': array([-1.01046513e-07,  1.04206826e-07, -3.57384364e-09,  5.14557922e-10,
         -1.31026447e-10,  4.53342372e-11,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.000

In [37]:
r_max = 22*BETA4

def I_1(pot, qi, qj):
    if np.abs(qi - qj) == 0:
        return quad(pot, 0, r_max)[0]
    else:
        return quad(pot, 0, r_max, weight='cos', wvar=qi-qj)[0]


def I_2(pot, qi, qj):
    return quad(pot, 0, r_max, weight='cos', wvar=qi+qj)[0]


def I_total(pot, qi, qj):
    return 1 / np.pi * 1/(qi*qj) * (I_1(pot, qi, qj) - I_2(pot, qi, qj))

In [38]:
I_total(pot, q[0], q[0]), I_total(pot, q[0], q[-1]), I_total(pot, q[-1], q[-1])

(64.90278011793957, 1.4392638315944353e-13, 0.06227702323414399)

In [31]:
sys = LocalSystem(R, qmesh=(q, wq, 20*2/R))

In [32]:
sys.v_tilde[0, 0] + sys.counterterm.gen(c0, 0)[0, 0]

64.90516651006948

In [33]:
sys.v_tilde[0, -1] + sys.counterterm.gen(c0, 0)[0, -1]

-1.4331198164279765e-07

In [34]:
sys.v_tilde[-1, -1] + sys.counterterm.gen(c0, 0)[-1, -1]

0.06227701172398386

In [27]:
%%time
v = np.array([[I_total(pot, qi, qj) for qi in q] for qj in q])

CPU times: user 1min 47s, sys: 232 ms, total: 1min 47s
Wall time: 1min 47s


In [29]:
v

array([[ 6.49029863e+01,  6.49030920e+01,  6.49036561e+01, ...,
        -1.22580440e-12,  2.95258358e-12, -1.58319021e-12],
       [ 6.49030920e+01,  6.49032017e+01,  6.49037557e+01, ...,
         5.47241218e-13, -2.04953886e-13,  0.00000000e+00],
       [ 6.49036561e+01,  6.49037557e+01,  6.49042634e+01, ...,
         5.95322635e-13, -2.38936866e-13,  2.99847574e-13],
       ...,
       [-1.22580440e-12,  5.47241218e-13,  5.95322635e-13, ...,
         6.25246863e-02,  6.24185654e-02,  6.23367243e-02],
       [ 2.95258358e-12, -2.04953886e-13, -2.38936866e-13, ...,
         6.24185654e-02,  6.23653982e-02,  6.23130324e-02],
       [-1.58319021e-12,  0.00000000e+00,  2.99847574e-13, ...,
         6.23367243e-02,  6.23130324e-02,  6.22770232e-02]])

In [28]:
np.linalg.norm((v - sys.v_tilde) / v)

/var/folders/_z/4zb077j96tz1_kb2vklbxydr0000gn/T/ipykernel_80058/3482098375.py:1: RuntimeWarning: divide by zero encountered in true_divide
  np.linalg.norm((v - sys.v_tilde) / v)


inf

In [29]:
with open(f'datfiles/matrix_elements/vtilde_R_{R:.8e}.txt', 'w+') as f:
    for i in range(nq):
        for j in range(nq):
            f.write(f'{wq[i]:.12e}  {q[i]:.12e}  {q[j]:.12e}  {v[i, j]:.12e}\n')

In [34]:
for i in tqdm(range(rgf.shape[0])):
    R, c0 = rgf[i]
    q, wq = log_mesh(0, 20*2/R, nq)
    
    def v4(r):
        if r == 0:
            return 0
        return potential(r, R)


    def pot(r):
        return v4(r) + c0*local_LO_counterterm(r, R)
    
    
    v = np.array([[I_total(pot, qi, qj) for qi in q] for qj in q])
    
    with open(f'datfiles/matrix_elements/vtilde_R_{R:.8e}.txt', 'w+') as f:
        for i in range(nq):
            for j in range(nq):
                f.write(f'{wq[i]:.16e}  {q[i]:.16e}  {q[j]:.16e}  {v[i, j]:.16e}\n')

100%|████████████████████████████████████████| 95/95 [2:48:45<00:00, 106.59s/it]
